<a href="https://colab.research.google.com/github/Sujana0710/cartoonModeration/blob/main/Anime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
from keras.preprocessing import image
import os
import tensorflow as tf

In [ ]:
# Data Preparation

# Data Preparation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.13
)

# Training Data Generator
train_generator = datagen.flow_from_directory(
    'Dataset1/train1',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # set as training data
)

# Validation Data Generator
validation_generator = datagen.flow_from_directory(
    'Dataset1/train1',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # set as validation data
)



In [ ]:
# Model Building

#The below code depicts the building of Proposed-3 Custom model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

# Model Checkpoint
checkpoint = ModelCheckpoint(
    'Custom.h5',
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=True
)
# CNN Model Architecture
model = Sequential()
# Convolutional Layers
model.add(Conv2D(64, (3, 3), input_shape=(256, 256, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fully Connected Layers
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))  # Assuming 3 classes: normal, violence, adultery
model.add(Activation('softmax'))



In [ ]:
# Compile the model
model.compile(optimizer='adam’,loss='categorical_crossentropy', metrics=['accuracy', f1_score])

# Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint]
)


In [ ]:
# Calculate percentages
if total_frames == 0:
    result = "None"
else:
    normal_percentage = (normal_count / total_frames) * 100
    violence_percentage = (violence_count / total_frames) * 100
    adultery_percentage = (adultery_count / total_frames) * 100
    result = f"Normal: {normal_percentage:.2f}%, Violence: {violence_percentage:.2f}%, Adultery: {adultery_percentage:.2f}%"

# Print or use the result as needed
print(result)
print("Total Frames: ",total_frames)
